In [1]:
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.layers import Flatten, Conv2D, MaxPooling2D
from keras.losses import sparse_categorical_crossentropy, mse
from keras.metrics import sparse_categorical_accuracy
from keras.optimizers import Adam
from keras import backend as K

from keras_utils import SampleMultinomial

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
num_classes = 10
batch_size = 100
epochs = 10
stats = []

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

('x_train shape:', (60000, 28, 28, 1))
(60000, 'train samples')
(10000, 'test samples')


In [56]:
x = Input(shape=input_shape)
h = Conv2D(32, (3, 3), activation='relu')(x)
h = Conv2D(64, (3, 3), activation='relu')(h)
h = MaxPooling2D(pool_size=(2, 2))(h)
h = Dropout(0.25)(h)
h = Flatten()(h)
h = Dense(128, activation='relu')(h)
h = Dropout(0.5)(h)
p = Dense(num_classes, activation='softmax')(h)
model = Model(x, p)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_59 (Dense)             (None, 128)               1179776   
__________

In [57]:
s = SampleMultinomial()(p)
s = s[:, np.newaxis]
#s = K.print_tensor(s, message="s: ")
#print("s.shape:", s)

h = Dense(128, activation='relu')(h)
b = Dense(1)(h)
#b = K.print_tensor(b, message="b: ")
#print("b.shape:", b)

def my_sparse_categorical_accuracy(y_true, y_pred):
    y_pred_cast = K.cast(y_pred, K.floatx())
    #y_pred_cast = K.print_tensor(y_pred_cast, message="y_pred_argmax_cast: ")
    y_equal = K.equal(y_true, y_pred_cast)
    #y_equal = K.print_tensor(y_equal, message="y_equal: ")
    y_equal_cast = K.cast(y_equal, K.floatx())
    #y_equal_cast = K.print_tensor(y_equal_cast, message="y_equal_cast: ")
    return y_equal_cast

def reinforce_loss(y_true, y_pred):
    r = my_sparse_categorical_accuracy(y_true, s)
    #r = K.print_tensor(r, message="r: ")
    #print("r.shape:", r)

    a = K.stop_gradient(r - b)
    #a = K.print_tensor(a, message="a: ")
    #print("a.shape:", a)

    return a * sparse_categorical_crossentropy(s, y_pred)[:, np.newaxis] + mse(r, b)

model.compile(loss=reinforce_loss, optimizer=Adam(lr=0.001), metrics=['sparse_categorical_accuracy'])

In [58]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)
#validation_data=(x_test, y_test))

Epoch 1/10
60000/60000 [==============================] - 81s - loss: 0.1040 - sparse_categorical_accuracy: 0.7482    
Epoch 2/10
60000/60000 [==============================] - 81s - loss: 0.0990 - sparse_categorical_accuracy: 0.9243    
Epoch 3/10
60000/60000 [==============================] - 81s - loss: 0.0963 - sparse_categorical_accuracy: 0.9426    
Epoch 4/10
60000/60000 [==============================] - 81s - loss: 0.0955 - sparse_categorical_accuracy: 0.9515    
Epoch 5/10
60000/60000 [==============================] - 82s - loss: 0.0907 - sparse_categorical_accuracy: 0.9561    
Epoch 6/10
60000/60000 [==============================] - 81s - loss: 0.0877 - sparse_categorical_accuracy: 0.9609    
Epoch 7/10
60000/60000 [==============================] - 81s - loss: 0.0891 - sparse_categorical_accuracy: 0.9630    
Epoch 8/10
60000/60000 [==============================] - 81s - loss: 0.0869 - sparse_categorical_accuracy: 0.9651    
Epoch 9/10
60000/60000 [========================

In [ ]:
npstats = np.array(stats)

plt.figure(figsize=(16, 6))

plt.subplot(1, 2, 1)
#plt.plot(npstats[:, 0])
plt.plot(npstats[:, 1])
plt.plot(npstats[:, 2])
#plt.legend(['total loss', 'policy gradient loss', 'baseline mse loss'])
plt.legend(['policy gradient loss', 'baseline mse loss'])
plt.title('loss')

plt.subplot(1, 2, 2)
plt.plot(npstats[:, 3])
plt.plot(npstats[:, 4])
plt.legend(['batch accuracy', 'baseline mean'])
#plt.legend(['batch accuracy'])
plt.title('accuracy')

In [59]:
score = model.evaluate(x_test, y_test, verbose=0)
#print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test accuracy:', 0.98680000000000001)
